In [1]:
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from string import punctuation
from collections import Counter

### Json model: 
```json
{
    "collections":[
      {
         "type":"detective",
         "url":"url",
         "text":"text" # выборка 30-50kb
      },
      {
          ...
      }
    ]
}
```

In [2]:
def read_file(file_path: str) -> str:
    """читает файл"""
    with open(file_path, "r") as f:
        content = f.read()
    return content

In [3]:
def read_stopwords(file_path: str) -> list:
    """возвращает список дополнительных стоп-слов"""
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    return lines

additional_stopwords = read_stopwords('./dataset/stopwords.txt')

In [4]:
detective = './dataset/detective.json'
fiction = './dataset/fiction.json'
parsed_detective = json.loads(read_file(detective))
parsed_detective = json.loads(read_file(fiction))

In [5]:
def tokenize_ru(file_text: str) -> list:
    """удаляет из текста символы пунктуации, стоп-слова"""
    # применим токенизацию
    tokens = word_tokenize(file_text)

    # удаляем пунктуационные символы
    tokens = [i for i in tokens if (i not in punctuation)]
    
    # чистим слова
    tokens = [i.lower().replace("«", "").replace("»", "").replace("`", "").replace("'", "").replace("\\", "") for i in tokens]
    tokens = list(filter(None, tokens))
    
    # удаляем стоп-слова
    stop_words = stopwords.words('russian')
    # stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...', '.', '-'])
    stop_words.extend(additional_stopwords)
    tokens = [i for i in tokens if (i not in stop_words)]

    return tokens

In [6]:
def normalize_dict(input_dict: dict) -> dict:
    """минимаксная нормализация словаря"""
    new_dict = {}
    min_val = min(input_dict.values())
    max_val = max(input_dict.values())
    for key, value in input_dict.items():
        try:
            new_dict[key] = (input_dict[key] - min_val) / (max_val - min_val)
        # если вхождение слова один раз значение равно 0
        except ZeroDivisionError:
            new_dict[key] = 0
    return new_dict

In [7]:
def dict_without_values_eq_zero(input_dict: dict) -> dict:
    """словарь без значений равных нулю"""
    return {k:v for k,v in input_dict.items() if v != 0}

In [8]:
def extract_feature(parsed_file: str) -> dict:
    """получаем вектор фич"""
    feature_dict: dict[int, dict] = {}

    for index, item in enumerate(parsed_file['collections'], start=1):
        # обработка текста
        tokenized_item = tokenize_ru(item['text']) # разбиваем по словам
        item_histogram = dict(Counter(tokenized_item)) # строим гистограмму слов
        normalized_histogram = normalize_dict(item_histogram) 
        dict_without_zero = dict_without_values_eq_zero(normalized_histogram) # удаляем слова по одному вхождению 
        dict_without_zero = {k: v for k, v in sorted(dict_without_zero.items(), key=lambda item: item[1], reverse=True)}
        # cобираем словарь фичей dict[index] = dict(feature)
        feature_dict[index] = dict_without_zero
    return feature_dict

feature_detective = extract_feature(parsed_detective)
feature_detective

{1: {'сказал': 1.0,
  'троп': 0.7692307692307693,
  'своей': 0.5384615384615384,
  'желе': 0.5384615384615384,
  'переводчик': 0.5,
  'пола': 0.46153846153846156,
  'капитан': 0.4230769230769231,
  'мог': 0.38461538461538464,
  'офицер': 0.38461538461538464,
  'мужчина': 0.34615384615384615,
  'боевой': 0.3076923076923077,
  'зефа': 0.2692307692307692,
  'понял': 0.23076923076923078,
  'дело': 0.23076923076923078,
  'глаза': 0.23076923076923078,
  'руки': 0.23076923076923078,
  'свой': 0.23076923076923078,
  'те': 0.23076923076923078,
  'я.': 0.23076923076923078,
  'вроде': 0.23076923076923078,
  'мама': 0.19230769230769232,
  'где-то': 0.19230769230769232,
  'корабль': 0.19230769230769232,
  'пол': 0.19230769230769232,
  'дара': 0.19230769230769232,
  'рема': 0.19230769230769232,
  'проектор': 0.19230769230769232,
  'тропа': 0.19230769230769232,
  'тримс': 0.19230769230769232,
  'говорила': 0.15384615384615385,
  'поэтому': 0.15384615384615385,
  'дарайцы': 0.15384615384615385,
  'оче

In [9]:
# собираем в один список все слова из feature_dict
all_words = []
for it in feature_detective.values():
    all_words.append(list(it.keys()))

# множество из уникальных слов во всем датасете
vocabulary = {item for sublist in all_words for item in sublist}

for word in vocabulary:
    for index, feature in feature_dict.items():
        if word not in feature.keys():
            feature[word] = 0    

NameError: name 'feature_dict' is not defined